# Weekly Activity

In [14]:
import sys
assert sys.version_info >= (3, 7)

import numpy as np
import cv2 as cv
from util_func import *

1. Experiment with different edge detectors: Sobel, Laplacian, Prewitt, Scharr derivatives and Canny operators (all with aperture size of 3) on image named 'pineapple.jfif'. Comment on the results.

In [2]:
img = cv.imread('images/pineapple.jfif', 0)

kernely = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
kernelx = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])


# Apply different edge detectors
sobel_x = cv.Sobel(img, cv.CV_64F, 1, 0, ksize=3)
sobel_y = cv.Sobel(img, cv.CV_64F, 0, 1, ksize=3)
laplacian = cv.Laplacian(img, cv.CV_64F, ksize=3)
prewitt_x = cv.filter2D(img, -1, kernelx)
prewitt_y = cv.filter2D(img, -1, kernely)
scharr_x = cv.Scharr(img, cv.CV_64F, 1, 0)
scharr_y = cv.Scharr(img, cv.CV_64F, 0, 1)
canny = cv.Canny(img, 100, 200)


cv.imshow('Original Image', img)
cv.imshow('Sobel X', sobel_x)
cv.imshow('Sobel Y', sobel_y)
cv.imshow('Laplacian', laplacian)
cv.imshow('Prewitt X', prewitt_x)
cv.imshow('Prewitt Y', prewitt_y)
cv.imshow('Scharr X', scharr_x)
cv.imshow('Scharr Y', scharr_y)
show_img('Canny', canny)

Sobel X: Vertical edges which contains high gradient is highlighted. Pineapple edges is not clearly shown due to the backgorund of land also high in gradient.

Sobel Y: Horizontal edges which contains high gradient is highlighted. Pineapple edges is not clearly shown due to the backgorund of land also high in gradient.

Laplacian: Almost every area with high intensity changes is highlighted. Messy edges formed.

Prewitt X: Exclude most noises occurs in Sobel X. Highlight edges horizontally.

Prewitt Y: Exclude most noises occurs in Sobel Y. Highlight edges horizontally.

Scharr X: Almost same with Sobel X but have more accurate edges in detailed parts

Scharr Y: Almost same with Sobel Y but have more accurate edges in detailed parts

Canny: Thin, fine edges formed. Remove noises.

2. Write a program to identify the white object (probably laptop) present in the image 'electronic.jfif'. Draw bounding boxes on the objects.

In [29]:
img = cv.imread('images/electronic.jfif')
blur = cv.GaussianBlur(img, (3, 3), 0)
gray = cv.cvtColor(blur, cv.COLOR_BGR2GRAY)
_, thresholded = cv.threshold(gray, 190, 255, cv.THRESH_BINARY)
contours, _ = cv.findContours(thresholded, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

for contour in contours:
    x, y, w, h = cv.boundingRect(contour)
    if w * h > 200:
        cv.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

show_img('img', img)

3. Isolate the clock with the aid of edge detection and contours' properties. The example result should be as follows:

In [2]:
def auto_canny(img, method, sigma=0.33):
    """Args:
    img: grayscale image
    method: median, otsu, triangle
    sigma: 0.33 (default)"""
    
    if method == "median":
        Th = np.median(img)
        
    elif method == "triangle":
        Th = cv.threshold(img, 0, 255, cv.THRESH_TRIANGLE)[0]
        
    elif method == "otsu":
        Th = cv.threshold(img, 0, 255, cv.THRESH_OTSU)[0]
        
    Thresh1 = (1-sigma) * Th
    Thresh2 = (1+sigma) * Th
    
    return cv.Canny(img, Thresh1, Thresh2)

In [17]:
img = cv.imread('images/clock.jpg')

factor = 300 / img.shape[1]
img = cv.resize(img, None, fx=factor, fy=factor)
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
blur = cv.bilateralFilter(gray, 7, 19, 13)
edge = auto_canny(blur, method="otsu")

contours, _ = cv.findContours(edge, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

# sort the contour with respect to contour area in descending order
contours = sorted(contours, key=cv.contourArea, reverse=True)[:5]

for c in contours:
    perimeter = cv.arcLength(c, True)
    area = cv.contourArea(c)
    circularity = 4 * np.pi * area / (perimeter * perimeter)
    if 0.7 < circularity < 1.3:
        screen = c
        break
        
img_copy = img.copy()
cv.drawContours(img_copy, [screen], -1, (0, 255, 2), 2)
show_img("contour", img_copy)
